In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy as sp
from scipy import stats
import sklearn as sk
from sklearn import model_selection

1. In this problem, we consider Random Forests. Implement the Random Forestalgorithm with 100 decision stumps. You must implement the decision stumps from scratch.Use information gain as the splitting measure. Apply your Random Forest implementationto the cancer dataset described above and do the following:
  *   Use m = 3 random attributes to determine the split of your decision stumps.  Learn amodel for an increasing number of decision stumps in the ensemble.  Compute the trainand test set classification error as the number of decision stumps increases.
  *   Vary the number of random attributes m from \\
  {2, . . . , p = 10} and fit a model using 100 decision stumps.  Compute the train and test set classification error as the numberof random featuresmincreases.



In [ ]:
def random_forest(X, y, learners, m):
  np.random.seed(5525)
  X_original = X
  y_original = y
  prediction_thresholds = []
  split_feature_indices = []
  signs = []
  iter_error = []
    
  for t in range(learners):
    index = np.random.choice(X.shape[0], X.shape[0], replace=True)
    D_t = X[index]
    y = y_original[index]
    feature_subset = get_feature_subset(X, m)
    entropy = []
    conditional_entropy = []

    for feature in range(feature_subset.shape[1]):
      entropy = np.append(entropy, get_entropy(feature_subset[:, feature])) 
      #input("Press Enter to continue...")
      conditional_entropy = np.append(conditional_entropy, get_conditional_entropy(feature_subset[:, feature], y))

    information_gain = get_information_gain(entropy, conditional_entropy)
    max_index = np.argmax(information_gain)
    split_feature_indices = np.append(split_feature_indices, max_index)
    split_feature = feature_subset[:,max_index]
    
    # Method 1: With Binary Classification
    #confusion = confusion_matrix(split_feature, y)
    #threshold = binary(split_feature, confusion, y)
    #sign = get_sign(split_feature, threshold, y)

    # Method 2
    threshold, sign = best_threshold(split_feature, y)
    
    prediction_thresholds = np.append(prediction_thresholds, threshold)
    signs = np.append(signs, sign)
    learner_consensus = _predict_one(threshold, split_feature, sign)
    error = np.sum(np.where(learner_consensus != y, 1, 0))/X.shape[0]
    # IF YOU WANT TO MUTE TRAINING CLASSIFICATION ERROR YOU CAN COMMENT THE 
    # PRINT CALL BELOW
    print("Classification error for stump", t+1, ": ", error)
    iter_error = np.append(iter_error, error)

  return prediction_thresholds, split_feature_indices, signs, iter_error


def get_feature_subset(X, m):
  cols = np.random.randint(0, X.shape[1], m)

  return X[:,cols]


def _predict_one(threshold, feature, sign):
  if sign == 1:
    predictions = np.where(feature >= threshold, 1, -1)
      #_ , counts = np.unique(predictions, return_counts=True)
      #votes = np.append(votes, counts[np.argmax(counts)])
    
  else:
    predictions = np.where(feature < threshold, 1, -1)

      #_ , counts = np.unique(predictions, return_counts=True)
      #votes = np.append(votes, counts[np.argmax(counts)])

  elements, counts = np.unique(predictions, return_counts= True)
  index = np.argmax(counts)

  return elements[index]


"""
 Each weak learner is going to generate (# of rows in the the data) votes. Basically, 1 votes per sample 
 in the data. Then we're going to have T weak learners, so we will have a (#Rows x T)
 matrix correspond to the votes of each learner for the given sample. Then
 we just take the consensus. 
"""
def predict(X, thresholds, feature_indices,signs):
  learner_votes = []
  learner_consensus = []
  
  for threshold, index, sign in zip(thresholds, feature_indices, signs):
    if sign == 1:
      predictions = np.where(X[:,int(index)] >= threshold, 1, -1)
      learner_votes = np.append(learner_votes, predictions)
      #_ , counts = np.unique(predictions, return_counts=True)
      #votes = np.append(votes, counts[np.argmax(counts)])
    
    else:
      predictions = np.where(X[:,index] < threshold, 1, -1)
      learner_votes = np.append(learner_votes, predictions)

      #_ , counts = np.unique(predictions, return_counts=True)
      #votes = np.append(votes, counts[np.argmax(counts)])

  learner_votes = np.reshape(learner_votes, (X.shape[0], signs.shape[0]))
  
  for row in learner_votes:
    elements, counts = np.unique(row, return_counts= True)
    index = np.argmax(counts)
    learner_consensus = np.append(learner_consensus, elements[index])

  return learner_consensus


def get_sign(feature, threshold, y):
  predictions = np.array([np.where(feature <  threshold, 1, -1),
                          np.where(feature >= threshold, 1, -1)])
  errors = []

  for guesses in predictions:
    error = 0
    
    for guess, label in zip(guesses, y):
      if guess != label:
        error += 1

    errors = np.append(errors, error/len(y))

  return np.argmin(errors) + 1

def get_entropy(feature):
  unique_elements, counts = np.unique(feature, return_counts=True)
  p_x = counts/np.sum(counts)

  return -(p_x.dot(np.log2(p_x)))


# Returns unsummed element entropys
def _get_entropys(p_x1, p_x2):
  
  return -(p_x1*np.log2(p_x1) + p_x2*np.log2(p_x2))

def get_conditional_entropy(feature, y):
  unique_y, counts_y = np.unique(y, return_counts=True)
  unique_x, counts_x = np.unique(feature, return_counts=True)
  confusion = confusion_matrix(feature, y)
  p_x = counts_x/np.sum(counts_x)
  joint_entropy = np.array([])

  for rows in confusion:
    p_x1 = rows[0]/len(feature)
    p_x2 = rows[1]/len(feature)
    
    if p_x1 == 0 or p_x2 == 0:
      joint_entropy = np.append(joint_entropy, 0)

    else:
      joint_entropy = np.append(joint_entropy, _get_entropys(p_x1, p_x1))

  conditional_entropy = p_x.dot(joint_entropy)

  return conditional_entropy

def get_information_gain(entropy, conditional_entropy):
  return entropy - conditional_entropy


def confusion_matrix(feature, y):
  unique_x, counts_x = np.unique(feature, return_counts=True)
  max = np.argmax(unique_x)
  confusion_positive = np.zeros(unique_x[max])
  confusion_negative = np.zeros(unique_x[max])
  
  for i,j in zip(feature, y):
    if j == 1:
      confusion_positive[i - 1] += 1

    elif j == -1:
      confusion_negative[i - 1] += 1
  
  confusion = np.vstack((confusion_positive, confusion_negative)).T

  return confusion[~np.all(confusion == 0, axis = 1)]


def binary(feature, confusion, y):
  unique_x, counts_x = np.unique(feature, return_counts=True)
  unique_y, counts_y = np.unique(y, return_counts=True)
  # {-1, 1}
  priors = counts_y/sum(counts_y)
  posterior = []
  
  for rows in confusion:
    p = priors[1]*(rows[0]/len(feature)) + priors[0]*(rows[1]/len(feature))
    posterior = np.append(posterior, p)

  threshold_index = unique_x[np.argmax(posterior)]
  
  return threshold_index

def best_threshold(feature, y):
  unique_x, counts_x = np.unique(feature, return_counts=True)
  unique_y, counts_y = np.unique(y, return_counts=True)
  # Sorts in ascending order
  sorted_elements = np.sort(unique_x)
  index = 1
  thresholds = []
  errors_above = []
  errors_below = []
  

  for i in range(len(sorted_elements) - 2):
    thresholds = np.append(thresholds, (sorted_elements[index - 1] + sorted_elements[index]) / 2 )
    index += 1

  for i in thresholds:
    # Value >= threshold --> -1
    predictions_above = np.where(feature >= i, 1, -1)
    error = np.where(predictions_above != y, 1, 0)
    errors_above = np.append(errors_above, np.sum(error)/len(y))
    # Value < threshold --> +1
    predictions_below = np.where(feature < i, 1, -1)
    error = np.where(predictions_below != y, 1, 0)
    errors_below = np.append(errors_below, np.sum(error)/len(y))

  min_above_index = np.argmin(errors_above)
  min_above_error = errors_above[min_above_index]
  min_below_index = np.argmin(errors_below)
  min_below_error = errors_below[min_below_index]

  # Below --> Sign 2
  if min_below_error < min_above_error:
    return thresholds[min_below_index], 2 

  # Above --> Sign 1
  else:
    return thresholds[min_above_index], 1


def main():
  np.set_printoptions(threshold=np.inf)
  plt.figure(figsize=(10,8))
  bc_data = np.loadtxt("/content/drive/MyDrive/breast-cancer-wisconsin-mode.csv", dtype=int,delimiter = ",")
  X = bc_data[:,1:-1]
  y = bc_data[:,-1]
  y = np.where(y == 2, 1, -1)
  X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size = 0.2, train_size = 0.8)
  training_errors = []
  test_errors = []

  # You can change the number of iterations and features here
  thresholds, features, signs, iter_error = random_forest(X_train, y_train, 100, 3)
  thresholds = thresholds.astype("int32")
  features = features.astype("int32")
  signs = signs.astype("int32")
  learner_consensus = predict(X_train, thresholds, features, signs)
  training_error = np.sum(np.where(learner_consensus != y_train, 1, 0))/X_train.shape[0]
  test_consensus = predict(X_test, thresholds, features, signs)
  error = np.sum(np.where(test_consensus != y_test, 1, 0))/X_test.shape[0]
  print("Test error: ", error)
  p = [2,3,4,5,6,7,8,9]

  for i in p:
    print()
    print("Varying feature subset, m = ", i, ":")
    thresholds, features, signs, iter_error = random_forest(X_train, y_train, 100, i)
    thresholds = thresholds.astype("int32")
    features = features.astype("int32")
    signs = signs.astype("int32")
    learner_consensus = predict(X_train, thresholds, features, signs)
    training_error = np.sum(np.where(learner_consensus != y_train, 1, 0))/X_train.shape[0]
    training_errors = np.append(training_errors, training_error)
    test_consensus = predict(X_test, thresholds, features, signs)
    error = np.sum(np.where(test_consensus != y_test, 1, 0))/X_test.shape[0]
    test_errors = np.append(test_errors, error)
    print("Test error: ", error)
    x = np.linspace(0, 100, num = 100)
    plt.plot(x, iter_error)
    plt.xlabel("Decision Trees")
    plt.ylabel("Classification Error")


  plt.show()

  plt.figure(figsize=(10,8))
  plt.plot(p, training_errors, color = "green")
  plt.plot(p, test_errors, color = "red")
  plt.xlabel("Random Features")
  plt.ylabel("Classification Errror")
  plt.legend(labels = ("Training Error", "Testing Error"))
  plt.show

if __name__ == "__main__":
  main()

